# Predictive Analysis of Instacart Order Data

# Table of Contents

* Introduction
* Data Analysis
    1. Datasets
    2. Order Analysis
    3. User Analysis
    4. Product Analysis
* Order Prediction


# Introduction

* Instacart is an American company that operates a grocery delivery and pick-up service. It is a two-sided marketplace which had a huge increase in its value due to COVID19.
* Through a prediction model, we can make better decision on the allocating the supply(deliverer), and provide better customer experience for the demand side.


# Analysis

* Deep dive on the dataset and the data structure. 

In [4]:
import numpy as np 
import pandas as pd   
import os  
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px
plt.rcParams["font.family"] = 'NanumSquare'
%matplotlib inline
color = sns.color_palette()
import warnings
import zipfile
warnings.filterwarnings('ignore')
print(zipfile.ZipFile('../input/instacart-market-basket-analysis/aisles.csv.zip'))

>>> ## 1. Dataset

In [5]:
aisles_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/aisles.csv.zip').open('aisles.csv'))
departments_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/departments.csv.zip').open('departments.csv'))
order_products__prior_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/order_products__prior.csv.zip').open('order_products__prior.csv'))
order_products__train_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/order_products__train.csv.zip').open('order_products__train.csv'))
orders_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/orders.csv.zip').open('orders.csv'))
products_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/products.csv.zip').open('products.csv'))

sample_submission = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/sample_submission.csv.zip').open('sample_submission.csv'))

### aisles : the aisles in which products are located at

In [6]:
print(aisles_data.head(5))

In [7]:
print(aisles_data.shape, '\n')
print(aisles_data.isna().sum())

In [8]:
# print(aisles_data.describe(), '\n')
# print(aisles_data.info())

### departments : higher level category of aisles. The category that each products belong to. 

In [9]:
print(departments_data.head(5))

In [10]:
print(departments_data.shape, '\n')
print(departments_data.isna().sum())

In [11]:
# print(departments_data.describe(), '\n')
# print(departments_data.info())

### order_products__prior : past order data of customers. 

In [12]:
print(order_products__prior_data.head(10))

In [13]:
print(order_products__prior_data.shape, '\n')
print(order_products__prior_data.isna().sum())

In [14]:
# print(order_products__prior_data.describe(), '\n')
# print(order_products__prior_data.info())

### <b>order_products__train</b> : Shares the same structure with order_products__prior. This has the latest order of a customer

In [15]:
print(order_products__train_data.head(10))

In [16]:
print(order_products__train_data.shape, '\n')
print(order_products__train_data.isna().sum())

In [17]:
# print(order_products__train_data.describe(), '\n')
# print(order_products__train_data.info())

### orders : Order data of customer. Each row contains an order. The latest order is split into train/test. Prior orders are labeled as 'prior'.

In [18]:
print(orders_data.head(12))

In [19]:
print(orders_data[orders_data['user_id'] == 2][['order_id','user_id', 'eval_set']].tail(), '\n')
print(orders_data[orders_data['user_id'] == 3][['order_id','user_id', 'eval_set']].tail(), '\n')
print(orders_data[orders_data['user_id'] == 4][['order_id','user_id', 'eval_set']].tail(), '\n')

In [20]:
print(orders_data.shape, '\n')
print(orders_data.isna().sum())
# na values on days_since_prior_order are user's first order.

In [21]:
# print(orders_data.describe(), '\n')
# print(orders_data.info())

### products : includes product name, its respective aisle and department

In [22]:
print(products_data.head(), '\n')

In [23]:
print(products_data.shape, '\n')
print(products_data.isna().sum())

In [24]:
# print(products_data.describe(), '\n')
# print(products_data.info())

>>> ## 2. Order Analysis

> ### All the orders are already labeled into 3 different labels: prior, train, test.  (train and test data are the most recent orders)

In [25]:
three_orders = orders_data.groupby('eval_set')["order_id"].aggregate("count").reset_index()
three_orders.rename(columns = {'eval_set':'order_type', "order_id":"count"}, inplace = True)
three_orders["ratio"] = three_orders['count'].apply(lambda x: x/three_orders["count"].sum()).astype(float)
print(three_orders)
plt.pie(three_orders.ratio,autopct='%.2f%%')
plt.title('Order Types')
plt.legend(loc='best', labels=three_orders['order_type'], fontsize='medium')
plt.show()


* #### Prior order data takes up around 94% of total orders. Out of 206209 total users, 131209 users' orders are in (order_products__train), and we need to predict the other 75000's order data, which are labeled as test dataset.

## **Number of Products per Order**


In [26]:
prior_products_per_order = order_products__prior_data.groupby('order_id')['add_to_cart_order'].aggregate('max').value_counts()
train_products_per_order = order_products__train_data.groupby('order_id')['add_to_cart_order'].aggregate('max').value_counts()

In [27]:
# Concantenate the most recent order and past orders 
all_order_products = pd.concat([order_products__prior_data, order_products__train_data], axis = 0)

In [28]:
products_per_order = all_order_products.groupby('order_id')['add_to_cart_order'].aggregate('max').value_counts()
plt.figure(figsize = (12,6))
plt.title('Number of Products per Order',fontsize = 15)
plt.bar(products_per_order.index, products_per_order.values)
plt.xlabel('Products per Order')
plt.ylabel('Count')
plt.xlim([0,80])
plt.xticks([x for x in range(0,81) if x%10 ==0])
plt.show()

In [29]:
print('%.2f %% of total orders had 12 or less products.' %(products_per_order[products_per_order.index<=12].sum() / products_per_order.sum()*100))
print('%.2f %% of total orders had 20 or less products.' %(products_per_order[products_per_order.index<=20].sum() / products_per_order.sum()*100))

* #### 7~8 products per order was the most frequent type of oders.
* #### Out of all the orders, 60% of them had 10 or less products, 90% had 20 or less products.

>>> ## 3. Customer Analysis

## Customer's Order Time

In [30]:
plt.figure(figsize = (12,8))
sns.countplot(x = orders_data['order_hour_of_day'], data = orders_data, color = color[9])
plt.xlabel('Order Hour of Day')
plt.ylabel('Count')
plt.title('Order Frequency by Hour of Day', fontsize =15)
plt.show()

In [31]:
plt.figure(figsize = (12,8))
sns.countplot(x = orders_data['order_dow'], data = orders_data, color = color[8])
plt.xlabel('Day of Week')
plt.ylabel('Count')
plt.title('Order Frequency by Day of Week', fontsize =15)
plt.show()

In [32]:
heatmap_data = orders_data.groupby(['order_dow', 'order_hour_of_day'])['order_number'].aggregate('count').reset_index()
heatmap_data = heatmap_data.pivot(index = 'order_dow', columns =  'order_hour_of_day',  values = 'order_number')
plt.figure(figsize = (12,8))
sns.heatmap(heatmap_data, cmap='YlGnBu')
plt.show()

* #### Satudays and Sundays have the highest order count. Also, most orders took place during 10AM and 4PM. 

## Order Frequency of Customers

In [33]:
from  matplotlib.ticker import FuncFormatter
plt.figure(figsize = (15,8))
sns.countplot(x = 'days_since_prior_order', data = orders_data)
plt.xlabel('Days since last Order') 
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: int(x)))
plt.ylabel("Count")
plt.title('Order Period', fontsize = 15)
plt.show()

* #### Most customers make orders once a month or once a week. 

## Order Number of Customers

In [34]:
order_per_user = orders_data.groupby('user_id')['order_number'].aggregate('count').reset_index()
order_per_user = order_per_user.sort_values(by = 'order_number') 
number_of_orders_made = order_per_user.groupby('order_number')['user_id'].aggregate('count').reset_index().rename(columns = {'user_id':'count'})
number_of_orders_made

In [35]:
plt.figure(figsize = (20,8))
sns.barplot(x = number_of_orders_made['order_number'].values, y = number_of_orders_made['count'], color = color[-1])
plt.xlabel('Number of Orders Made by User',size = 12) 
plt.xticks(rotation=-25, size = 9)
plt.ylabel("User Count",size = 12)
plt.title('Number of Orders Made by Users', fontsize = 15)
plt.show()

In [36]:
print(number_of_orders_made[(number_of_orders_made['order_number'] <= 20)].sum()['count']/number_of_orders_made.sum()['count'])
print(number_of_orders_made[(number_of_orders_made['order_number'] <= 10)].sum()['count']/number_of_orders_made.sum()['count'])
print('50% of customers made less than 10 orders.')
print('75% of customers made less than 20 orders.')

>>> ## 4. Product Analysis

### High Demand Products

In [37]:
# Total number of orders for each products
high_order_demand = all_order_products.groupby("product_id")['order_id'].aggregate('count').reset_index().rename(columns={'order_id':'order_count'})

# Merge products name and products id table 
high_order_demand = pd.merge(products_data[['product_id','product_name']], high_order_demand, how = 'left', on=['product_id'])

# Order by order count
high_order_demand = high_order_demand.sort_values(by = 'order_count', ascending = False)
high_order_demand[:20]

In [38]:
plt.figure(figsize = (14,7))
sns.barplot(x = high_order_demand[:15]['product_name'].values, y = high_order_demand[:15]['order_count'])
plt.xlabel('Product Name',size = 12) 
plt.xticks(rotation=-45, size = 12)
plt.ylabel("Order Count",size = 12)
plt.title('Most Ordered Products', fontsize = 15)
plt.show()

* #### Regular groceries such as fruits, onions and milk have the highest order frequency. 
* #### Out of 20 most frequently ordered products, 15 of them are organic. 

## Products with the most reorders

In [39]:
high_reorder_demand = all_order_products.groupby('product_id')['reordered'].aggregate({'count','sum'}).reset_index()
high_reorder_demand = high_reorder_demand.rename(columns = {'count': 'order_count', 'sum':'reorder_count'})
high_reorder_demand['reorder_ratio'] = high_reorder_demand['reorder_count'] / high_reorder_demand['order_count']
high_reorder_demand = pd.merge(products_data[['product_id','product_name']], high_reorder_demand, how = 'left', on=['product_id'])
high_reorder_demand = high_reorder_demand.sort_values(by = 'reorder_count', ascending = False)

# Not much difference from order-frequency, thus commented the code below

# plt.figure(figsize = (18,7))
# sns.barplot(x = high_reorder_demand[:15]['product_name'], y = high_reorder_demand[:15]['reorder_count'])
# plt.xlabel('Product Name',size = 12) 
# plt.xticks(rotation=325, size = 12)
# plt.ylabel("Reorder Count",size = 12)
# plt.title('Most Reordered Products', fontsize = 15)
# plt.show()

high_reorder_demand[:10]

## Most reordered products (with at least 10000 reorders)

In [ ]:
high_reorder_ratio = high_reorder_demand.sort_values(by = 'reorder_ratio', ascending = False)
high_reorder_ratio = high_reorder_ratio[high_reorder_ratio['reorder_count'] > 10000] 

plt.figure(figsize = (18,7))
sns.barplot(x = high_reorder_ratio[:25]['product_name'], y = high_reorder_ratio[:25]['reorder_ratio'])
plt.xlabel('Product Name',size = 12) 
plt.xticks(rotation=90, size = 12)
plt.ylabel("Reorder Ratio",size = 12)
plt.title('Products with High Reorder Ratio (>100 reorders)', fontsize = 15)
plt.show()

high_reorder_ratio[:20]


* #### Products with low order count or reorder count are products that have low overall demand but have loyal customers. Therefore, I decided to add the condition (reorder_count > 10000).
* #### Milk is one of the most reordered products. This may be due to the fact that there are various types of milk(low-fat, skim, almond, etc) and usually people tend to consume the milk they are used to consuming. 
* #### Fruits and vegetables have high order and reorder count, but it can be attributed to the fact that they do not have different brands competing against one another. Reorder ratio of these products are lower than that of milk and water. 

## Reorder Ratio based on the order of cart addition

In [41]:
cart = pd.merge(all_order_products, high_reorder_demand[['product_id', 'reorder_ratio']], on = ['product_id'])
cart = cart.groupby('add_to_cart_order')['reorder_ratio'].aggregate('mean').reset_index()

In [42]:
plt.figure(figsize=(12,8))
sns.pointplot(cart['add_to_cart_order'][:60], cart['reorder_ratio'][:60], color = color[0])
plt.title('Average Reorder Ratio by Add-to-Cart Order', fontsize=15)
plt.xlabel('add_to_cart_order',fontsize=12)
plt.ylabel('reorder_ratio',fontsize=12)
plt.show()

* #### The sooner a customer adds a product to a cart, the higher its reorder ratio. People would normally buy regular groceries and add what comes to their mind first. Regular groceries can be seen as must-have groceries and therefore, this trend is realistic.  

## Department and Aisle Analysis 

In [45]:
tree = products_data.groupby(['department_id', 'aisle_id',])['product_name'].aggregate('count').reset_index()
tree = pd.merge(aisles_data[['aisle_id','aisle']], tree, how = 'left', on=['aisle_id'])
tree = pd.merge(departments_data[['department_id', 'department']], tree, how = 'left', on = ['department_id'])
 

fig = px.treemap(tree, path = ['department','aisle'], title = 'Department & Aisles Structure', width=1400, height = 1000)
fig.update_layout(font = dict(size = 14))
fig.show()

In [46]:
# Create a comprehensive dataframe which includes department, aisle, order count, reorder count/ratio data of all the products
df = pd.merge(products_data, departments_data, how='left')
df = pd.merge(df, aisles_data, how= 'left', on = ['aisle_id'])
sold = all_order_products.groupby('product_id')['add_to_cart_order'].aggregate('count').reset_index().rename(columns = {'add_to_cart_order':'orders'}).sort_values(by = 'orders',ascending = False)
df = pd.merge(df, sold, on=['product_id'])
df = pd.merge(df, high_reorder_demand[['product_id','reorder_count','order_count','reorder_ratio']], on=['product_id'])
df = df.drop(columns = ['orders'], axis=1)
df

## Sales per Department

In [47]:
dep_sold = df.groupby('department')['order_count'].aggregate('sum').reset_index().sort_values(by = 'order_count', ascending = False)
dep_sold['order_share'] = dep_sold['order_count'].apply(lambda x: x/dep_sold['order_count'].sum())
dep_sold.head()

In [44]:
plt.figure(figsize=(14,14))
plt.pie(dep_sold['order_share'], labels= dep_sold['department'], autopct='%1.2f%%')
plt.title('Order Share by Department', fontsize=15)
plt.show()


* #### Produce and dairy products had the highest order count. As we have seen earlier, most customers order weekly and therefore bulk products have the lowest demand.

## Reorder Rate per Department

In [ ]:
df2 = df[['aisle_id','reorder_ratio','department']]
cols = list(df2.groupby('department')['reorder_ratio'].aggregate('mean').reset_index().sort_values(by='reorder_ratio', ascending = False)['department']) 
plt.figure(figsize=(24,8))
sns.boxplot(x='department', y='reorder_ratio', data = df2, order= cols)
plt.title('Reorder Rate by Department', fontsize =15)
plt.xlabel('Department', fontsize=12)
plt.xticks(fontsize =12)
plt.ylabel('Reorder_Ratio', fontsize=12)
plt.xticks(rotation=45)
plt.show()

* #### It is worth noting that bakery and pets department did not have high order count, but have high reorder ratio. Pantry department has a low reorder rate considering its order count.
* #### Some outliers such as personal care department has order count less than 100.

In [ ]:
# df[(df['department']=='pantry')|(df['department']=='personal care')].sort_values(by = 'reorder_ratio', ascending=False).head(20)

## Sales per Aisle(excluding bulk, other, missing)

In [ ]:
aisle_sold = df.groupby(['department','aisle'])['order_count'].aggregate('sum').reset_index().sort_values(by = 'order_count', ascending = False)
aisle_sold['order_share'] = aisle_sold['order_count'].apply(lambda x: x/aisle_sold['order_count'].sum())

lst = ['bulk','other','missing']
aisle_sold = aisle_sold[~aisle_sold['department'].isin(lst)] 

fig, axs = plt.subplots(6,3, figsize = (30,48))
fig.subplots_adjust(hspace = 1) 

for x,plot in zip(aisle_sold.groupby('department'), axs.flatten()):
    detail = x[1]
    sns.barplot(detail['aisle'], detail['order_count'], ax = plot)
    plot.set_ylabel('Order Count', fontsize = 12)
    plot.set_xticklabels(detail['aisle'], rotation=45, fontsize =12)
    plot.set_title(x[0], fontsize = 12)


In [ ]:
aisle_reorder_ratio = df.groupby(['department','aisle'])['reorder_ratio'].aggregate('mean').reset_index().sort_values(by = 'reorder_ratio', ascending = False)
lst = ['bulk','other','missing']
aisle_reorder_ratio = aisle_reorder_ratio[~aisle_reorder_ratio['department'].isin(lst)] 

fig, axs = plt.subplots(6,3, figsize = (30,48))
fig.subplots_adjust(hspace = 1) 

for x,plot in zip(aisle_reorder_ratio.groupby('department'), axs.flatten()):
    detail = x[1]
    sns.barplot(detail['aisle'], detail['reorder_ratio'], ax = plot)
    plot.set_ylabel('Reorder Ratio', fontsize = 12)
    plot.set_ylim([0, 0.7])
    plot.set_xticklabels(detail['aisle'], rotation=45, fontsize =12)
    plot.set_title(x[0], fontsize = 12)

# **Order Prediction**

### Train the model on train data and predict the products that will be in a new order

In [ ]:
print(dir())
for variable in dir():
    if '_' not in ['In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'exit', 'get_ipython', 'quit']:
        del globals()[variable]
print(dir())

import numpy as np 
import pandas as pd   
import os  
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px
plt.rcParams["font.family"] = 'NanumSquare'
%matplotlib inline
color = sns.color_palette()
import warnings
import zipfile
warnings.filterwarnings('ignore')
print(zipfile.ZipFile('../input/instacart-market-basket-analysis/aisles.csv.zip'))

aisles_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/aisles.csv.zip').open('aisles.csv'))
departments_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/departments.csv.zip').open('departments.csv'))
order_products__prior_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/order_products__prior.csv.zip').open('order_products__prior.csv'))
order_products__train_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/order_products__train.csv.zip').open('order_products__train.csv'))
orders_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/orders.csv.zip').open('orders.csv'))
products_data = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/products.csv.zip').open('products.csv'))

sample_submission = pd.read_csv(zipfile.ZipFile('../input/instacart-market-basket-analysis/sample_submission.csv.zip').open('sample_submission.csv'))

## Data Preprocessing

In [1]:
# Comprehensive data frame with all the information on products 
products = products_data.copy()
products['orders'] = order_products__prior_data.groupby('product_id')['order_id'].aggregate('count').reset_index()['order_id'].astype(int)
products['reorders'] = order_products__prior_data.groupby('product_id')['reordered'].aggregate('sum').reset_index()['reordered'].astype(int)
products['reorder_ratio'] = products['reorders'] / products['orders'].astype(float)
products.set_index('product_id', drop=False, inplace=True)
del aisles_data
del departments_data

products.head()

In [ ]:
# Merged orders_data on prior data on order_id 
priors = pd.merge(order_products__prior_data, orders_data, how='left', on = 'order_id')
orders_data.set_index('order_id', inplace=True, drop=False)

priors.head()

In [ ]:
# Dataframe with all the information about user
users = pd.DataFrame()
users['user_id'] = orders_data.groupby('user_id').size().index
users['total_orders'] = orders_data.groupby('user_id').size().values
users['period_between_orders'] =orders_data.groupby('user_id')['days_since_prior_order'].aggregate('mean').reset_index()['days_since_prior_order'].astype(np.float32)
users['products_bought_count'] = priors.groupby('user_id')['product_id'].apply(len).reset_index()['product_id']
users['unique_products_bought_count'] = priors.groupby('user_id')['product_id'].apply(set).apply(len).reset_index()['product_id']
users['products_bought'] = priors.groupby('user_id')['product_id'].apply(set).reset_index()['product_id']
users['products_per_order'] = users['products_bought_count'] / users['total_orders']
users.set_index('user_id', inplace = True, drop = True)

users.head()

In [ ]:
# Dataframe with a customer and her purchased products 
# Created a new index combining product_id and user_id
userXproduct = priors.copy()
userXproduct['user_product'] = userXproduct['product_id'] + userXproduct['user_id'] * 100000
userXproduct = userXproduct.sort_values('order_number')
userXproduct = userXproduct.groupby('user_product', sort=False).aggregate({'order_id': ['size', 'last'], 'add_to_cart_order': 'mean', 'reordered':'sum'})
userXproduct.columns = ['total_orders', 'last_order_id', 'mean_add_to_cart_order', 'reordered_count']
userXproduct['reorder_ratio'] = userXproduct['reordered_count'] / userXproduct['total_orders']
userXproduct = userXproduct.astype({'total_orders': np.int16, 'last_order_id': np.int32, 'mean_add_to_cart_order': np.int16, 'reordered_count':np.int16})
del priors

userXproduct.head()

In [ ]:
# No need to split the data into train set and test set as they are already labeled
test_orders = orders_data[orders_data['eval_set'] == 'test']
train_orders = orders_data[orders_data['eval_set'] == 'train']
order_products__train_data.set_index(['order_id', 'product_id'], inplace=True, drop=False)

In [ ]:
train_orders.head(5)

In [ ]:
order_products__train_data.head(5)

## Features that will be used as model inputs

### Extract features related to user's purchase, product and user.

In [48]:
def features(orders_input, labels_given = False):
    print('start')
    
    order_list, product_list,labels = [], [], []
    for row in orders_input.itertuples():
        order_id = row.order_id
        user_id = row.user_id
        
        user_products = users['products_bought'][user_id]
        product_list += user_products
        order_list += [order_id] * len(user_products)
        if labels_given:
            labels += [(order_id, product) in order_products__train_data.index for product in user_products]
        
    df = pd.DataFrame({'order_id':order_list, 'product_id':product_list}, dtype=np.int32)
    labels = np.array(labels, dtype=np.int8)
    
#     product related feature
    # aisle id
    df['aisle_id'] = df['product_id'].map(products['aisle_id'])
        #     df['department_id'] = df['product_id'].map(products['department_id']) 
    df['product_orders'] = df['product_id'].map(products['orders'])
        #     df['product_reorders'] = df['product_id'].map(products['reorders'])
    df['product_reorder_ratio'] = df['product_id'].map(products['reorder_ratio'])
    
    
#     user related feature
    
    df['user_id'] = df['order_id'].map(orders_data['user_id'])
    df['user_total_orders'] = df['user_id'].map(users['total_orders'])
    df['user_total_products_bought_count'] = df['user_id'].map(users['products_bought_count'])
    df['user_unique_products_bought_count'] = df['user_id'].map(users['unique_products_bought_count'])
    df['user_period_between_orders'] = df['user_id'].map(users['period_between_orders'])
    df['products_per_order'] =  df['user_id'].map(users['products_per_order'])    
        #     df['dow'] = df['order_id'].map(orders_data['order_dow'])
    df['hod'] = df['order_id'].map(orders_data['order_hour_of_day'])
    df['days_since_prior_order'] = df['order_id'].map(orders_data['days_since_prior_order']) 
    
    
#     'user and purchased product' related feature
    # new index combining user_id and product id
    df['user_product_index'] = df['user_id'] * 100000 + df['product_id']
    df['user_product_orders'] = df['user_product_index'].map(userXproduct['total_orders'])
    df['user_product_order_ratio'] = (df['user_product_orders'] / df['user_total_orders']).astype(np.float32)
    df['user_product_add_to_cart_order'] = df['user_product_index'].map(userXproduct['mean_add_to_cart_order'])
    df['user_product_reorder_ratio'] = df['user_product_index'].map(userXproduct['reorder_ratio'])
    df['user_product_order_term'] = df['user_total_orders'] - (df['user_product_index'].map(userXproduct['last_order_id'])).map(orders_data['order_number'])       

    print('finished')
    
    return (df, labels)

In [ ]:
df_train , labels = features(train_orders, labels_given = True)

## Training the Model

In [ ]:
import lightgbm as lgbm
df_train.drop(axis = 1, inplace = True, columns = ['order_id', 'product_id', 'user_id', 'user_product_index'])

# Final model input
model_train_set = lgbm.Dataset(df_train, label=labels, categorical_feature=['aisle_id','hod'], free_raw_data=False)

In [ ]:
params = { 
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 100,
    'max_depth': 15
}

model = lgbm.train(params, model_train_set, 150)

In [ ]:
lgbm.plot_importance(model, figsize = (9,9))

## Future Order Prediction

In [ ]:
df_test, label_test = features(test_orders)

test_set = df_test.drop(axis = 1, columns = ['order_id', 'product_id', 'user_id', 'user_product_index']) 

pred = model.predict(test_set)

df_test['pred'] = pred
d = {}
user_pred_basket = {}

i = 0
for row in df_test.itertuples(): 
    i += 1
    if i % 1000000 == 0:
        print('working on row :', i, ' out of 4833292')
        
#     user_ppo = round(row.products_per_order)
#     if row.user_id not in user_pred_basket.keys():
#         user_pred_basket[row.user_id] = sorted(df_test[df_test['user_id'] == row.user_id].pred)[::-1][:user_ppo]

##############################################
# sum = 0
# count = 0
# for key in user_pred_basket.keys():
#     count +=1
#     sum += min(user_pred_basket[key])
# print(sum / count)  -> 0.23029478294378136
##############################################

# On average, if pred value is above .23, a customer will order the product. 
# This will ensure that total products in the order matches the number of products in the user's average order.

    if row.pred > .23:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

for order in test_orders.order_id:
    if order not in d:
        d[order] = 'None'
print('done')

In [ ]:
my_submission = pd.DataFrame.from_dict(d, orient='index')
my_submission.reset_index(inplace=True)
my_submission.columns = ['order_id', 'products']
my_submission.head(5)

In [ ]:
# my_submission.to_csv('submit4.csv', index=False)

## Findings and Points of Improvement
* Water, milk and other typical groceries such as bread had high reorder ratios from the model prediction as well. Out of all the feature inputs, aisle_id had the highest importance. 
* Each customer seems to have their unique purchase pattern that they follow each order. This can be seen from the high importance of (user_unique_products_bought_count) variable that the model returns. This pattern would be stronger for products such as water and milk, which varies in brand. 
* After training the model and checking each variable's importance, I have removed certain variables that had minimal effect on the output. 
* Aisle is a subcategory of department and therefore the department_id was collinear with aisle_id. The prediction score of the model improved after removing such duplicate variable.

* Additional feature engineering to simplify, or further break down certain products maybe helpful. (i.e. aggregating skim, low-fat, almond milks into milk)
